# 4.0 可视化演示

本 notebook 用于演示 3D 可视化功能。

## 目标
1. 静态 3D 渲染
2. 呼吸动画
3. 对比可视化

In [ ]:
import sys
sys.path.append('..')

import numpy as np
from pathlib import Path

# 检查 PyVista
try:
    import pyvista as pv
    print(f"PyVista 版本: {pv.__version__}")
    
    # 设置 Jupyter 后端
    pv.set_jupyter_backend('static')
except ImportError:
    print("PyVista 未安装，请运行: pip install pyvista")

## 1. 创建测试数据

In [ ]:
def create_synthetic_lung():
    """创建合成肺部数据"""
    shape = (64, 64, 64)
    ct = np.full(shape, -1000, dtype=np.float32)
    
    # 创建椭球形肺部
    center = np.array(shape) // 2
    for z in range(shape[0]):
        for y in range(shape[1]):
            for x in range(shape[2]):
                # 椭球方程
                dist = ((z - center[0])/20)**2 + ((y - center[1])/25)**2 + ((x - center[2])/15)**2
                if dist < 1:
                    ct[z, y, x] = -700 + np.random.randn() * 30
    
    return ct

def create_synthetic_lesion(lung_ct):
    """创建合成病灶"""
    mask = np.zeros_like(lung_ct, dtype=np.uint8)
    
    # 在肺部区域内添加几个病灶
    lesion_centers = [(35, 35, 35), (25, 40, 30), (40, 25, 35)]
    
    for cz, cy, cx in lesion_centers:
        r = np.random.randint(3, 6)
        for z in range(max(0, cz-r), min(64, cz+r)):
            for y in range(max(0, cy-r), min(64, cy+r)):
                for x in range(max(0, cx-r), min(64, cx+r)):
                    if (z-cz)**2 + (y-cy)**2 + (x-cx)**2 <= r**2:
                        if lung_ct[z, y, x] > -900:  # 只在肺部区域内
                            mask[z, y, x] = 1
    
    return mask

In [ ]:
# 创建测试数据
lung_ct = create_synthetic_lung()
lesion_mask = create_synthetic_lesion(lung_ct)

print(f"肺部 CT 形状: {lung_ct.shape}")
print(f"病灶体素数: {lesion_mask.sum()}")

## 2. 静态 3D 渲染

In [ ]:
from src.visualization.static_render import create_lung_mesh, create_lesion_mesh

# 创建网格
lung_mesh = create_lung_mesh(lung_ct, threshold=-800)
lesion_mesh = create_lesion_mesh(lesion_mask)

print(f"肺部网格: {lung_mesh.n_points} 点, {lung_mesh.n_cells} 面")
print(f"病灶网格: {lesion_mesh.n_points} 点, {lesion_mesh.n_cells} 面")

In [ ]:
# 渲染
plotter = pv.Plotter()
plotter.set_background('white')

# 添加肺部（半透明灰色）
plotter.add_mesh(lung_mesh, color='gray', opacity=0.3, smooth_shading=True)

# 添加病灶（红色）
plotter.add_mesh(lesion_mesh, color='red', opacity=0.8, smooth_shading=True)

plotter.add_axes()
plotter.camera_position = 'xy'
plotter.show()

## 3. 多角度视图

In [ ]:
# 创建多视图
plotter = pv.Plotter(shape=(1, 3), window_size=(1200, 400))

views = ['xy', 'xz', 'yz']
titles = ['前视图', '顶视图', '侧视图']

for i, (view, title) in enumerate(zip(views, titles)):
    plotter.subplot(0, i)
    plotter.add_mesh(lung_mesh, color='gray', opacity=0.3)
    plotter.add_mesh(lesion_mesh, color='red', opacity=0.8)
    plotter.add_text(title, font_size=12)
    plotter.camera_position = view

plotter.show()

## 4. 呼吸动画预览

In [ ]:
from src.visualization.dynamic_breath import apply_breathing_deformation

# 显示不同呼吸阶段
plotter = pv.Plotter(shape=(1, 4), window_size=(1600, 400))

phases = [0, np.pi/2, np.pi, 3*np.pi/2]
phase_names = ['呼气末', '吸气中', '吸气末', '呼气中']

for i, (t, name) in enumerate(zip(phases, phase_names)):
    plotter.subplot(0, i)
    
    deformed_lung = apply_breathing_deformation(lung_mesh, t, amplitude=0.1)
    deformed_lesion = apply_breathing_deformation(lesion_mesh, t, amplitude=0.1)
    
    plotter.add_mesh(deformed_lung, color='gray', opacity=0.3)
    plotter.add_mesh(deformed_lesion, color='red', opacity=0.8)
    plotter.add_text(name, font_size=12)
    plotter.camera_position = 'xy'

plotter.show()

## 5. 保存渲染结果

In [ ]:
# 保存截图
output_dir = Path('../data/04_final_viz/renders')
output_dir.mkdir(parents=True, exist_ok=True)

plotter = pv.Plotter(off_screen=True, window_size=(1920, 1080))
plotter.set_background('white')
plotter.add_mesh(lung_mesh, color='gray', opacity=0.3)
plotter.add_mesh(lesion_mesh, color='red', opacity=0.8)
plotter.camera_position = 'xy'

screenshot_path = output_dir / 'demo_render.png'
plotter.screenshot(str(screenshot_path))
plotter.close()

print(f"截图已保存: {screenshot_path}")

## 6. 总结

本 notebook 演示了:
1. 使用 PyVista 进行 3D 体渲染
2. 肺部 + 病灶双通道可视化
3. 呼吸动画的不同阶段
4. 保存高分辨率渲染结果